# Data Engineer Associate Exam - Virtual Reality Fitness

ActiVR provides a virtual reality device designed for exercise and fitness.

ActiVR offers a range of products, including VR devices and subscription-based fitness programs through their apps.

The sales team at ActiVR wants to analyze user data to enhance their marketing strategy and evaluate their products. For this, it is crucial that the data is clean, accurate, and available for reporting.

They need your assistance in preparing the data before launching a new promotional campaign.


**Database Schema**

The data schema for ActiVR's database is outlined as follows:

- **events:** Contains records of events registered in different games.
- **games:** Stores information about various games available on the platform.
- **devices:** Holds data about the virtual reality devices used by the users.
- **users:** Contains details about the users utilizing the ActiVR platform.

![schema](schema.png)


# Task 1

ActiVR's sales team wants to use the information it has about users for targeted marketing.

However, they suspect that the data may need to be cleaned before.

The expected data format and types for the `users` table according to the sales team's requirements is shown in the table below.

Write an SQL query that returns the `users` table with the specified format. Ensure that your query does not modify the `users` table.


| Column Name     | Description                                                  |
|-----------------|--------------------------------------------------------------|
| user_id         | Unique integer (assigned by the database, cannot be altered). Missing values are not possible due to the database structure. |
| age             | Integer representing the age of the customer. Missing values should be replaced with the average age.                  |
| registration_date | Date when the user made an account first (YYYY-MM-DD). Missing values should be replaced with January 1st, 2024.   |
| email | Email address of the user. Missing values should be replaced with Unknown.  |
| workout_frequency        | Workout frequency as a lowercase string, one of: _minimal_, _flexible_, _regular_, _maximal_. Missing values must be replaced with _flexible_. |


In [18]:
-- select * from users;
-- with clean_data as(
-- select 
-- cast(user_id as integer) as user_id,
-- cast(age as integer) as age,
-- cast(registration_date as date) as registration_date,
-- cast(email as text) as email,
-- lower(workout_frequency) as workout_frequency
-- from users),
-- avg_age AS (
--     SELECT AVG(age) AS avg_age
--     FROM clean_data
--     WHERE age IS NOT NULL
-- )


-- update clean_data
-- set age=case when age is null then avg_age else age end,
-- registration_date=case when registration_date is null then '2024-01-01' else registration_date end,
-- email=case when email is null then 'Unknown' else email end,
-- workout_frequency=case when workout_frequency is null then flexible else workout_frequency end;







-- select * from clean_data;
-- Create a CTE to calculate average age
-- Step 1: Create a Temporary Table



-- WITH transformed_data AS (
--     SELECT 
--         CAST(user_id AS INTEGER) AS user_id,
--         CAST(age AS INTEGER) AS age,
--         CAST(registration_date AS DATE) AS registration_date,
--         CAST(email AS TEXT) AS email,
--         LOWER(workout_frequency) AS workout_frequency
--     FROM users
-- ),
-- avg_age AS (
--     SELECT AVG(age) AS avg_age
--     FROM transformed_data
--     WHERE age IS NOT NULL
-- ),
-- clean_data AS (
--     SELECT 
--         user_id,
--         CASE 
--             WHEN age IS NULL THEN (SELECT avg_age FROM avg_age)
--             ELSE age 
--         END AS age,
--         CASE 
--             WHEN registration_date IS NULL THEN '2024-01-01'::DATE
--             ELSE registration_date 
--         END AS registration_date,
--         CASE 
--             WHEN email IS NULL THEN 'Unknown'
--             ELSE email 
--         END AS email,
--         CASE 
--             WHEN workout_frequency IS NULL THEN 'flexible'
--             ELSE workout_frequency 
--         END AS workout_frequency
--     FROM transformed_data
-- )
-- SELECT * FROM clean_data;



WITH clean_data AS (
    SELECT 
        CAST(user_id AS INTEGER) AS user_id,
        -- Replace missing age values with the average age
        CASE 
            WHEN age IS NULL THEN (SELECT AVG(age)::INTEGER FROM users WHERE age IS NOT NULL)
            ELSE age 
        END AS age,
        -- Replace missing registration_date with January 1, 2024 (correctly cast to DATE)
        CASE 
            WHEN cast(registration_date AS DATE) IS NULL THEN '2024-01-01'::DATE
            ELSE cast(registration_date AS DATE)
        END AS registration_date,
        -- Replace missing email with "Unknown"
        CASE 
            WHEN cast(email AS TEXT) IS NULL THEN 'Unknown'
            ELSE cast(email AS TEXT)
        END AS email,
        -- Handle NULL and empty workout_frequency, replace with 'flexible'
        CASE 
            WHEN workout_frequency IS NULL OR workout_frequency = '' THEN 'flexible'
            ELSE LOWER(cast(workout_frequency AS TEXT))
        END AS workout_frequency
    FROM users
)
SELECT * FROM clean_data;






user_id  age  ...                    email workout_frequency
0          1   56  ...         hi_1@example.com          flexible
1          2   46  ...      hello_2@myemail.com           minimal
2          3   32  ...        hello_3@email.com           maximal
3          4   60  ...          user4@email.com           maximal
4          5   25  ...           hi_5@email.com           minimal
..       ...  ...  ...                      ...               ...
345      346   48  ...  contact_346@example.com          flexible
346      347   52  ...    contact_347@email.com           minimal
347      348   50  ...   contact348@example.com          flexible
348      349   38  ...          hi349@email.com           regular
349      350   49  ...       hi_350@myemail.com           maximal

[350 rows x 5 columns]

# Task 2
It seems like there are missing values in the `events` table for the column `game_id` for all events before the year 2021.

However, we know that before 2021 there were only games where the `game_type` is `running`. The `game_id` for these games can be found in the `games` table.

Write a query so that the `events` table has a `game_id` for all events including those before 2021.

In [19]:
-- WITH events_with_game_id AS (
--     SELECT 
--         e.event_id, 
--         COALESCE(e.game_id, g.game_id) AS game_id,  -- Replace NULL game_id with the game_id from games table
--         e.event_date,
--         e.game_type,
--         e.email -- Include other necessary fields from events
--     FROM events e
--     LEFT JOIN games g ON e.game_type = g.game_type
--     WHERE e.event_date < '2021-01-01' 
--       AND e.game_type = 'running'
-- )
-- SELECT e.event_id, e.game_id, e.event_date, e.game_type, e.email  -- Adjust columns as needed
-- FROM events e
-- LEFT JOIN events_with_game_id ewg ON e.event_id = ewg.event_id;

-- select * from events;


WITH events_with_game_id AS (
	select event_id, 
	case when cast(game_id as integer) is null then 4 else cast(game_id as integer) end as game_id,
	device_id,
	user_id, 
	event_time
	from events
)
	
select * from events_with_game_id;
	

event_id  game_id  device_id  user_id          event_time
0            1        3          4       73 2021-06-11 02:07:04
1            2        3          5      141 2023-05-28 16:15:07
2            3        4          1       70 2023-08-31 13:28:50
3            4        4          2      262 2020-06-18 17:50:41
4            5        1          2      340 2021-01-21 06:34:48
...        ...      ...        ...      ...                 ...
1495      1496        4          1      225 2022-12-17 22:48:18
1496      1497        1          2      333 2022-04-19 03:50:09
1497      1498        1          3       42 2021-10-13 02:10:40
1498      1499        1          1      303 2022-07-01 09:38:06
1499      1500        2          5      246 2023-02-27 01:47:40

[1500 rows x 5 columns]

# Task 3

ActiVR's sales team plans to launch a promotion for upgrades to virtual reality devices.

They aim to target customers who have participated in events related to specific game types.

Write a SQL query to provide the `user_id` and `event_time` for users who have participated in events related to `biking` games.


In [20]:
with event_biking as(
	SELECT e.user_id, e.event_time
	FROM events e
	JOIN games g ON e.game_id = g.game_id
	WHERE g.game_type = 'biking'
)
select * from event_biking;


user_id          event_time
0        340 2021-01-21 06:34:48
1         49 2021-10-17 08:53:15
2        109 2021-12-19 16:01:28
3        216 2023-03-16 14:57:29
4        339 2021-01-02 04:51:58
..       ...                 ...
293      266 2021-07-30 08:18:42
294      296 2023-02-28 10:16:18
295      333 2022-04-19 03:50:09
296       42 2021-10-13 02:10:40
297      303 2022-07-01 09:38:06

[298 rows x 2 columns]

# Task 4

After running their promotion, the sales team at ActiVR wants to investigate the results.

To do so, they require insights into the number of users who participated in events for each `game_type`.

Write a SQL query that returns the count of unique users for each game type `game_type` and `game_id`. The user count should be shown in a column `user_count`.

In [21]:
with users_game as(
SELECT 
    g.game_type, 
    e.game_id, 
    COUNT(DISTINCT e.user_id) AS user_count
FROM events e
JOIN games g ON e.game_id = g.game_id
GROUP BY g.game_type, e.game_id
)
select * from users_game;

game_type  game_id  user_count
0    biking        1         198
1   dancing        3         193
2    rowing        2         205
3   running        4         202